# Simple color mapping for spans

In [1]:
def value_mapper_discrete(segment, attribute, value_mapping, default_value, conflict_value):
    
    if segment[2].length() != 1:
        return conflict_value
    
    return value_mapping.get(segment[2][0].get_attr(attribute), default_value)    

In [8]:
def value_mapper_discrete(segment, attribute, value_mapping, default_value, conflict_value) -> str:
    """Function for applying mappings. Arguments are: segment - the same segment that is passed to the visualisers;
    attribute - the attribute of the span of that segment (e.g. "lemma"); value_mapping - dictionary that maps the
    attribute values to the values given to css (e.g. {"kala":"red"}); default_value - value to be returned if
    the attribute value is not in value_mapping; conflict_value - value to be returned if there is an overlapping
    span. Returns the value for the css element."""
    """Does not work for spans with ambigious annotations"""
    if len(segment[1]) != 1:
        return conflict_value

    return value_mapping.get(getattr(segment[1][0],attribute), default_value)

In [ ]:
def my_best_color_mapper(segment):
    return value_mapper_discrete(segment, "lemma", {"kala":"red"}, "blue", "green")

In [36]:
def my_best_color_mapper(segment):
    return value_mapper_discrete(segment, "root", {'mis':"red"}, "blue", "green")

In [1]:
from estnltk import Text

In [25]:
tekst = Text("Tee kas mis")
tekst.tag_layer()

Text(text='Tee kas mis')

In [26]:
tekst.morph_analysis

Layer(name=morph_analysis, spans=SL[AS[Annotation(Tee, {'clitic': '', 'ending': '0', 'form': 'sg n', 'lemma': 'tee', 'partofspeech': 'S', 'root': 'tee', 'root_tokens': ('tee',)})],
AS[Annotation(kas, {'clitic': '', 'ending': '0', 'form': '', 'lemma': 'kas', 'partofspeech': 'D', 'root': 'kas', 'root_tokens': ('kas',)})],
AS[Annotation(mis, {'clitic': '', 'ending': '0', 'form': 'pl n', 'lemma': 'mis', 'partofspeech': 'P', 'root': 'mis', 'root_tokens': ('mis',)}), Annotation(mis, {'clitic': '', 'ending': '0', 'form': 'sg n', 'lemma': 'mis', 'partofspeech': 'P', 'root': 'mis', 'root_tokens': ('mis',)})]])

In [27]:
from estnltk.visualisation.span_visualiser.fancy_span_visualisation import DisplaySpans

display = DisplaySpans()

In [28]:
display.span_decorator.color_mapping= my_best_color_mapper

In [37]:
display(tekst.morph_analysis)

Tee kas mis

In [38]:
tekst.morph_analysis[2].root

,root
0,mis
1,mis


Analogous mappings can be defined for other style elements
* backgrpund color
* font
* ....

## A decorator for drawing spans without attribute values

There are two ways to do styling
* Direct through styling attributes
* Indirect through id and class values

These are mutually exclusive in our setting to avoid WTF experiences

In [2]:
class SpanDecorator:
    # call
    # css
    pass

In [3]:
class DirectPlainSpanVisualiser(SpanDecorator):
    
    default_conf_colour = "red"
    
    def __init__(colour_mapping=None, bg_mapping, ..., fill_empty_spans=False):
        self.bg_mapping = bg_mapping or default_bg_mapping
        #self.event_attacher = event_attacher
    
    def __call__(self, segment):
        
        # Simple text no span to fill
        if not fill_empty_span and is_pure_text(segment):
            return cut(text,segment)
    
        # There is a span to decorate
        output += '<span style='
        if self.colour_mapping is not None:
            output += 'color:' + self.color_mapping(segment)
        output += 'background:'+ self.bg_mapping(segment)
        output += '>'
        
        output += segment[0]
        output += '</span>'

        return output

    def default_bg_mapping(segment):
        if segment[2].length() != 1:
            return default_conf_colour
    
        return "yellow"    

SyntaxError: invalid syntax (<ipython-input-3-82d8f09a1dfc>, line 3)

In [ ]:
class IndirectPlainSpanVisualiser(SpanDecorator):
    
    def __init__(class_mapper, id_mapper, event_attacher, fill_empty_spans = False ):    
        pass
    
    def __call__(self, segment):
        
        # Simple text no span to fill
        if not fill_empty_span and is_pure_text(segment):
            return cut(text,segment)
    
        # There is a span to decorate
        output += '<span '
        if id_mapper:
            output = 'id = ' + id_mapper(segment)

        if class_mapper:
            output = 'class = ' + class_mapper(segment)

        output += cut(text, segment)
        output += '</span>'

        return output

# Attribute visualiser

In [ ]:
class DirectAttributeVisualiser(SpanDecorator):
    
    def __init__(colour_mapping, bg_mapping, ..., event_attacher, fill_empty_spans=False):
        pass
    
    def __call__(self, segment):
        
        # Simple text no span to fill
        if not fill_empty_span and is_pure_text(segment):
            return cut(text,segment)
    
        # There is a span to decorate
        output += '<span style='
        output += 'color:' + color_mappint(segment)
        output += 'background:'+ bg_mapping(segment)
        
        # This attaches events
        output += event_attacher(segment)

        output += '>'
        
        output += cut(text, segment)
        
        output += '</span>'

        return output
    
    def css():
        return 'css'
        
    def event_handler_code():
        return 'event handling code in JS'

In [ ]:
class Display:    

    css = 'mingi css'
    span_decorator = smth    

    def __init__(self, ??):
        pass

    def __call__(self):        
        seqments = decompose_to_elementary_spans(Text, layers)

        output = '' 
        for segment in segments:
            output += sd.decorate(segment) 

        return output # html string

In [ ]:
class FancySpanVisualisation:
    
    def _call_

In [ ]:
class DisplayAttributes:
    
    sd=DirectPlainSpanVisualiser()
    
    def __call__(self, layer):
        seqments = decompose_to_elementary_spans(layer)

        output = '' 
        for segment in segments:
            output += sd.decorate(segment)    

        return output # html string
    

In [2]:
from estnltk import Text
from estnltk.fancy_span_visualisation import DisplayAttributes 


display = DisplayAttributes()
display = FancySpanVisualisation.dispaly_spans
display = FancySpanVisualisation.validate_spans

t = Text('bla')

display(text)

ImportError: No module named 'estnltk.fancy_span_visualisation'

In [1]:
from estnltk.FancySpanVisualisation import DisplayAttributes as Display

span_decorator = DirectPlainSpanVisualiser(color_mapper=lambda segment: color_mapper_discrete(segment, concrete value))

display = Display(decorator=span_decorator)

display(t)

SyntaxError: invalid syntax (<ipython-input-1-0e3724b2ec75>, line 3)

In [ ]:
t.layer_name

In [ ]:
t.display

In [ ]:
print(t)
print(t)